In [1]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

In [2]:
from datasets import load_dataset

dataset_ori = load_dataset("text", data_files=['human3.fna.line.bpe'])
dataset_ori

/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2944826
    })
})

In [3]:
dataset_select = dataset_ori["train"].shuffle(seed=42).select(range(1000000))
dataset_select

Dataset({
    features: ['text'],
    num_rows: 1000000
})

In [4]:
tokenizer = Tokenizer(models.Unigram()) #vocab (List[Tuple[str, float]], optional) – A list of vocabulary items and their relative score [(“am”, -0.2442),…]
#models.Unigram()， 可以用一个字典初始化

In [5]:
# from tokenizers import Regex

# tokenizer.normalizer = normalizers.Sequence(
#     [
#         normalizers.Replace("``", '"'),
#         normalizers.Replace("''", '"'),
#         normalizers.NFKD(),
#         normalizers.StripAccents(),
#         normalizers.Replace(Regex(" {2,}"), " "),
#     ]
#)

In [6]:
#tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()#此预标记器用提供的替换字符替换任何空格。然后它尝试在这些空间上进行分裂

In [7]:
#tokenizer.pre_tokenizer.pre_tokenize_str("Let's test the pre-tokenizer!")

In [8]:
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]
trainer = trainers.UnigramTrainer(
    vocab_size=50000, special_tokens=special_tokens, unk_token="<unk>",max_piece_length=12
)
def get_training_corpus(dataset_select):
    for i in range(0, len(dataset_select), 1000):
        yield dataset_select[i : i + 1000]["text"]

import time

start_time = time.time()
tokenizer.train_from_iterator(get_training_corpus(dataset_select), trainer=trainer)
end_time = time.time()

print("time cost s:", int(end_time-start_time))
#tokenizer.model = models.Unigram()
#tokenizer.train(["human3.fna.line.bpe"], trainer=trainer)

encoding = tokenizer.encode("GGGGCTGGAGAAGGGGAGAAGAGGAAAGTGAGGTTGCCT")
print(encoding.tokens)
tokenizer.save("tokenizer8.json")



time cost s: 86382
['GGGGCT', 'GGAGAAGGGG', 'AGAAGAGGAAA', 'GTGAGGT', 'TGCCT']


In [9]:
encoding = tokenizer.encode("GGGGCTGGAGAAGGGGAGAAGAGGAAAGTGAGGTTGCCT")
print(encoding.tokens)

['GGGGCT', 'GGAGAAGGGG', 'AGAAGAGGAAA', 'GTGAGGT', 'TGCCT']


In [10]:
tokenizer.save("tokenizer8.json")